**Instructions**

- This question is in two parts

- The first part guides you through implementing a timestepper for a system of two coupled nonlinear ODEs

- The second part guides you through implementing the adjoint method for this problem

- Throughout this assessment, try to demonstrate your understanding of the course material by
    - commenting **briefly** on the results you obtain as you go
    - commenting **briefly** on any choices you are making
    - making comments in your code
    - following best practices

- We are not looking for long or overly-detailed answers. Rather, we are looking for concise observations which demonstrate your understanding of the course material

- Read the text for each question and follow the instructions carefully

- Do NOT change the name of this file

- Make sure to execute all your cells and save the result of the execution. We will only mark cells that have been executed and will not execute any cells ourselves.

In [ ]:
import numpy as np
import scipy.linalg as sl
from matplotlib import pyplot as plt

## Question 1 [30 Marks]: A nonlinear timestepping problem
---

In this question, we consider a problem consisting of 2 variables $u$ and $v$, which obey the following coupled ODEs:

$$
    \begin{align}
    \frac{d u}{d t} &= A + u^2 v - (B+1) u, \\
    \frac{d v}{d t} &= Bu -u^2 v.
    \end{align}
$$

These equations are known as the Brusselator, and they describe the dynamics of a particular chemical reaction between two agents, whose quantities are given by $u$ and $v$, with $A$ and $B$ being reaction rates.

We will use the vector notation

$$
    {\bf u} = \begin{pmatrix} u \\ v \end{pmatrix}.
$$

Using an implicit timestepper with a time step $\Delta t$, the system is then described by

$$
    \frac{{\bf u}^{(i+1)} - {\bf u}^{(i)}}{\Delta t} =  {\bf R}({\bf u}^{(i+1)}),
$$

where the superscript indicates the time step index, and

$$
    {\bf R}({\bf u}) = \begin{pmatrix} A + u^2 v - (B+1) u \\ Bu -u^2 v \end{pmatrix}.
$$

Throughout this exercise, we will use $A=1$, $B=3$, and $\Delta t = 0.05$.

In [ ]:
A = 1
B = 3

dt = 0.05

**Question 1.1** [2 marks]

Write a Python function to implement ${\bf R}({\bf u})$

In [ ]:
def R(u):
    

To solve this problem using the nonlinear methods (e.g. Newton) which we studied during the lectures, we will also require the Jacobian

$$
    R'({\bf u}) = \begin{pmatrix}
        \frac{\partial R_1}{\partial u}, \frac{\partial R_1}{\partial v} \\
        \frac{\partial R_2}{\partial u}, \frac{\partial R_2}{\partial v}
    \end{pmatrix}.
$$

**Q1.2** [4 marks]

Write a Python function to implement the Jacobian matrix ${R'}({\bf u})$, and test it.

Hint: test each component of $\bf R$ separately.

In [ ]:
def Rprime(u):
    

**Q1.3** [4 marks]

The single-timestep problem can be written in the form ${\bf F}({\bf u}^{(i+1)}, {\bf u}^{(i)}) = 0$, where
$$
{\bf F}({\bf u}^{(i+1)}, {\bf u}^{(i)}) = {\bf u}^{(i+1)} - {\bf u}^{(i)} - \Delta t \,\, {\bf R}({\bf u}^{(i+1)}).
$$

For the purposes of solving this problem at a particular time step, we are solving for ${\bf u}^{(i+1)}$, with ${\bf u}^{(i)}$ already known. The relevant Jacobian is therefore ${\bf F}' = \frac{\partial {\bf F}({\bf u}^{(i+1)}, {\bf u}^{(i)})}{\partial {\bf u}^{(i+1)}}$. Implement $\bf F$ and ${\bf F}'$ below, making use of the `R` and `Rprime` functions written above.

In [ ]:
def F(u_iplus1, u_i):
    

def Fprime(u_iplus1):
    

**Q1.4** [12 marks]

Calculate the Jacobian $F'$ for a few different values of $u$ and $v$ in the range 0--5. Is it SPD?

Briefly comment on the applicability/suitability of the following solving strategies for solving the single-timestep problem. Limit your responses to 2-3 sentences for each point.

 - Steepest descent
 - Newton's method with line search
 - Gauss-Newton
 - Quasi-Newton secant methods
 - Truncated Newton

**Q1.5** [6 marks]

Implement a timestepping function which solves the nonlinear problem at each timestep, based on the code below. Use the call to `scipy` provided to perform the nonlinear solve; see the scipy documentation for more detail on usage. Choose a suitable initial guess for each solve, and explain your choice. The function should return a list or array of ${\bf u}^{(i)}$ at all timesteps $i$, including the initial condition $i=0$.

``` python
from scipy.optimize import root

def timestepper(u_initial, n_steps):
    #
    #
    for i in range(n_steps):
        #
        #
        result = root(fun=..., jac=..., x0=..., method='hybr')
        #
        #
    return ...
```

**Q1.6** [2 marks]

Run the timestepper for 200 timesteps, from an initial condition of ${\bf u} = (1.01, 3.01)^T$, and plot the trajectory with $u$ on the $x$-axis and $v$ on the $y$-axis.

## Question 2 [30 Marks]: The adjoint method
---

In the second half of this exercise, we will use the adjoint method to estimate the initial condition ${\bf u}^{(0)}$ which produced an observed final state $\hat{\bf u}$, after $n$ timesteps. Specifically, we will take $n=50$ and $\hat{\bf u}$ as given below.

In [ ]:
n = 50
uhat = np.array([0.87075507, 2.4519925])

Assuming a suitable functional $f$ that depends only on the final model state ${\bf u}^{(n)}$, the adjoint equation for this discrete problem takes the following form:

$$
  \begin{pmatrix}
    \underline{\mathbf I} & -\underline{\mathbf I} \\
    & F'({\bf u}^{(1)})^T & -\underline{\mathbf I} \\
    & & F'({\bf u}^{(2)})^T & -\underline{\mathbf I} \\
    & & & \ddots & \ddots \\
    & & & & F'({\bf u}^{(n-1)})^T & -\underline{\mathbf I} \\
    & & & & & F'({\bf u}^{(n)})^T
  \end{pmatrix}
  \begin{pmatrix}
    \boldsymbol{\lambda}^{(0)} \\ \boldsymbol{\lambda}^{(1)} \\ \boldsymbol{\lambda}^{(2)} \\ \vdots \\ \boldsymbol{\lambda}^{(n-1)} \\ \boldsymbol{\lambda}^{(n)}
  \end{pmatrix}
=
  \begin{pmatrix}
    0 \\ 
    0 \\ 
    0 \\ 
    \vdots \\
    0 \\ 
    \frac{\partial f}{\partial {\bf u}^{(n)}} \\ 
  \end{pmatrix},
$$

where $F'({\bf u})$ is the same Jacobian you have already derived in part 1 above.

**Q2.1** [4 marks]

Write down a suitable functional $f({\bf u}^{(n)})$ for this problem which takes its minimum value when ${\bf u}^{(n)} = \hat{\bf u}$. Also derive the corresponding $\frac{\partial f}{\partial {\bf u}^{(n)}}$.

You may write these using LaTeX, as a Python function, or written by hand with a legible photograph of your equations embedded here.

**Q2.2** [6 marks]

Complete the function below to solve the adjoint equation for $\lambda^{(0)}$ by iterating backwards in time, given a corresponding set of forward model variables ${\bf u}^{(i)}$.

In [ ]:
def solve_adjoint_eqn(forward_soln, uhat):
    df_du_n = ... # Based on your choice of functional from Q2.1
    
    # Solve the bottom row of the adjoint equation
    ...
    
    # Iterate backwards through the middle rows
    ...
    
    # Solve top row of adjoint equation for lam_0
    ...
    
    return ...

Assuming again that $f$ depends only on the final model state, the gradient of the reduced functional $\hat{f}$ with respect to the model's initial condition ${\bf u}^{(0)}$ is then given by

$$
    \frac{d \hat{f}}{d {\bf u^{(0)}}} = \lambda^{(0)}.
$$

**Q2.3** [5 marks]

Implement $\hat{f}$ and $\frac{d \hat{f}}{d {\bf u^{(0)}}}$ as Python functions, and test the gradient function.

In [ ]:
def fhat(m):
    ...

def dfhat_dm(m):
    ...

**Q2.4** [5 marks]

Explain why BFGS is an appropriate choice for solving this inversion problem, and suggest at least one other appropriate method from the lectures, with justification.

Solve the problem using the code below and check the result. Start from an initial guess of `(1, 3)`. Display the result.

```python
from scipy.optimize import minimize
result = minimize(fun=..., x0=..., jac=..., method='BFGS')
```

**Q2.5** [5 marks]

Consider the following variations on this inversion problem and explain how you would adapt your approach for each (you don't have to write or run any more code!):
 - The parameters $A$ and $B$ are also unknown
 - You know the initial condition exactly, but wish to calculate the sensitivity of ${\bf u}^{(i)}$ to the initial condition ${\bf u}^{(0)}$, at all timesteps $i$

**Q2.6** [5 marks]

Experiment with the model and consider various initial conditions. Plot some examples, and describe the behaviour you observe. Comment on the sensitivity of the long-term behaviour with respect to the initial condition, and what implications this has on inversion problems of this kind.